In [1]:
# !pip install elasticsearch

In [2]:
# log_path = "./00323-M/run2/log-1006.txt"

# # 1. Read the file line by line and save lines from 96861 onwards
# with open(log_path, 'r') as file:
#     lines = [line for idx, line in enumerate(file, start=1) if idx >= 96861]

# # 2. Reopen the file in write mode and overwrite with the saved lines
# with open(log_path, 'w') as file:
#     file.writelines(lines)

In [3]:
# can't multiply sequence by non-int of type 'str'from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import warnings
# warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
import re
from datetime import datetime
from datetime import datetime, timedelta
from dataQuery import SendQuery

In [33]:
es_username = "xindong.x"
es_password = "Azerty123!"

# run = 3
# if run == 3:
#     id_machine = "00007"
#     startTime = "2023-08-25T18:00:00+02:00"
#     stopTime  = "2023-08-27T10:21:00+02:00"
#     log_path  = "./00007_btl/run3/log.txt"
# elif run == 4:
#     pass

# # weekend 1006-1008
# id_machine = "00323"
# startTime = "2023-10-06T06:00:00+02:00"
# stopTime  = "2023-10-09T09:30:00+02:00"
# log_path  = "./00323-M/run2/log-1006.txt"

# # weekend 1013-1016
# id_machine = "00323"
# startTime = "2023-10-13T18:15:00+02:00"
# stopTime  = "2023-10-15T18:30:00+02:00"
# log_path  = "./00323-M/run5/log.txt"


# weekend 10120-1022
id_machine = "00323"
startTime = "2023-10-20T15:45:00+02:00"
stopTime  = "2023-10-20T18:30:00+02:00"
log_path  = "./00323-M/run10/log.txt"

id_machine = "00322"
startTime = "2023-10-20T16:30:00+02:00"
stopTime  = "2023-10-20T17:00:00+02:00"
log_path  = "./00322-G/run6/log.txt"
# dead depuis 1020 1657

# weekend 10120-1022
id_machine = "00007"
startTime = "2023-10-25T14:30:00+02:00"
stopTime  = "2023-10-25T16:10:00+02:00"
log_path  = "./00007_btl_new/demo/run1/log.txt"
log_path  = "./00007_btl_new/demo/run2/log.txt"

data_session = SendQuery(es_username, es_password, id_machine, startTime, stopTime, es_index = "stream-reporting-sessions", size=1000)
data_packing = SendQuery(es_username, es_password, id_machine, startTime, stopTime, es_index = "stream-reporting-packings", size=1000)

In [34]:
data_session

,machineSerialNumber,sessionStart-1s,sessionStop,nbBottle
session_id,,,,
1,00007,2023-10-25 14:28:05.400917,2023-10-25 14:45:51.198759,123
2,00007,2023-10-25 15:03:24.840468,2023-10-25 15:05:31.524822,13
3,00007,2023-10-25 15:09:35.490933,2023-10-25 15:15:51.391732,16
4,00007,2023-10-25 15:22:47.052485,2023-10-25 15:32:08.843835,32
5,00007,2023-10-25 15:32:25.985160,2023-10-25 15:41:15.741005,49
6,00007,2023-10-25 15:46:36.015266,2023-10-25 15:52:15.869763,34
7,00007,2023-10-25 15:52:45.520391,2023-10-25 16:01:53.559428,60
8,00007,2023-10-25 16:02:05.103782,2023-10-25 16:05:40.141601,27


In [35]:
data_session.nbBottle.sum()

354

In [36]:
def AddSessionId(row):
    mask = (data_session['sessionStart-1s'] <= row['actionTime']) & (data_session['sessionStop'] >= row['actionTime'])
    session_ids = data_session.index[mask]
    return int(session_ids[0]) if len(session_ids) == 1 else None

data_packing['session_id'] = data_packing.apply(AddSessionId, axis=1)
data_packing['session_id'].fillna(-1, inplace=True)
data_packing['session_id'] = data_packing['session_id'].astype(int)

data_packing = data_packing[['machineSerialNumber', 'session_id', 'actionTime', 'isAccept', 'barCode', 'brand', 'reason']]

In [37]:
data_packing.head()

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason
packing_id,,,,,,,
1,00007,1,2023-10-25 14:30:21.826135,True,3256223100141,U,
2,00007,1,2023-10-25 14:30:25.864491,True,3254380008430,Cristaline,
3,00007,1,2023-10-25 14:30:30.892241,True,3664320011510,Quezac,
4,00007,1,2023-10-25 14:30:36.122441,True,3664320011510,Quezac,
5,00007,1,2023-10-25 14:30:43.465418,True,3256223100141,U,


In [38]:
data_packing[data_packing['reason'] == 'ExcludeList']

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason
packing_id,,,,,,,


In [39]:
data_packing['isAccept'].value_counts()

True     354
False      8
Name: isAccept, dtype: int64

In [40]:
print(data_packing['actionTime'].min())
print(data_packing['actionTime'].max())

2023-10-25 14:30:21.826135
2023-10-25 16:09:50.720798


In [41]:
data_packing.loc[data_packing['session_id'] == -1]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason
packing_id,,,,,,,
342,00007,-1,2023-10-25 16:05:53.406921,True,3068320110097,Badoit,
343,00007,-1,2023-10-25 16:05:56.813399,True,3068320110097,Badoit,
344,00007,-1,2023-10-25 16:06:06.413410,True,7613035866386,Contrex,
345,00007,-1,2023-10-25 16:06:15.783100,True,3068320110097,Badoit,
346,00007,-1,2023-10-25 16:06:20.617281,True,3068320110097,Badoit,
347,00007,-1,2023-10-25 16:06:23.512725,True,3068320110097,Badoit,
348,00007,-1,2023-10-25 16:06:30.027766,False,3068320110097,Badoit,Refus AC
349,00007,-1,2023-10-25 16:06:56.467880,False,5038862166508,Innocent,Refus AC
350,00007,-1,2023-10-25 16:07:25.520125,False,7613035866386,Contrex,Refus AC


In [42]:
data_packing.groupby(by = ["session_id"])["isAccept"].sum()

session_id
-1     15
 1    105
 2     13
 3     17
 4     33
 5     50
 6     34
 7     60
 8     27
Name: isAccept, dtype: int64

#### Import log file

In [43]:
with open(log_path, 'r') as file:
    log_data = file.read()
    
in_matches  = re.findall(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\,\d{3}).*?count numbers in  (\d+)', log_data)
out_matches = re.findall(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\,\d{3}).*?count numbers out (\d+)', log_data)

data_ia_dict = {}
for match in in_matches:
    timestamp = datetime.strptime(match[0], '%Y-%m-%d %H:%M:%S,%f')
    if timestamp <= data_packing['actionTime'].max() and data_packing['actionTime'].min() <= timestamp:
        data_ia_dict[timestamp] = True

for match in out_matches:
    timestamp = datetime.strptime(match[0], '%Y-%m-%d %H:%M:%S,%f')
    if timestamp <= data_packing['actionTime'].max() and data_packing['actionTime'].min() <= timestamp:
        data_ia_dict[timestamp] = False

In [44]:
data_ia = pd.DataFrame(list(data_ia_dict.items()), columns=['passlineTime', 'isIn_Out'])
data_ia = data_ia.sort_values(by='passlineTime', ascending=True)
data_ia.reset_index(drop=True, inplace=True)
data_ia.index += 1
data_ia.index.name = 'count_id'

data_ia.head(5)

,passlineTime,isIn_Out
count_id,,
1,2023-10-25 14:30:25.458,True
2,2023-10-25 14:30:30.481,True
3,2023-10-25 14:30:35.708,True
4,2023-10-25 14:30:43.072,True
5,2023-10-25 14:30:48.585,True


In [45]:
len(data_ia), len(data_ia[data_ia['isIn_Out']==True]) - len(data_ia[data_ia['isIn_Out']==False])

(329, 323)

#### Merge according to actionTime

In [46]:
# df = pd.merge_asof( data_packing.sort_values("actionTime"), 
#                     data_ia.sort_values("actionTime"), 
#                     on="actionTime", 
#                     tolerance=pd.Timedelta("2s"),
#                     direction="nearest")
# df.head(40)

In [76]:
def between2Packing(start, stop, data_dict):
    count_id_list = [count for time, count in data_dict.items() if start <= time <= stop]
    if len(count_id_list) == 0:
        return 0
    else:
        return len(count_id_list)

def between2Packing_addtime(start, stop, in_dict, out_dict):
    inline  = [time for time, count in in_dict.items()  if start <= time <= stop]
    outline = [time for time, count in out_dict.items() if start <= time <= stop]
    
    if len(inline) == 0:
        if len(outline) == 0:
            return 0
        return outline[0]
    else:
        return inline[0]

In [77]:
with open(log_path, 'r') as file:
    log_data = file.read()
    
in_matches  = re.findall(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\,\d{3}).*?count numbers in  (\d+)', log_data)
out_matches = re.findall(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\,\d{3}).*?count numbers out (\d+)', log_data)

in_dict  = {datetime.strptime(match[0], '%Y-%m-%d %H:%M:%S,%f'): int(match[1]) for match in in_matches}
out_dict = {datetime.strptime(match[0], '%Y-%m-%d %H:%M:%S,%f'): int(match[1]) for match in out_matches}

In [78]:
# in_dict  = {datetime.strptime(match[0], '%Y-%m-%d %H:%M:%S,%f') + timedelta(seconds=1): int(match[1]) for match in in_matches}
# out_dict = {datetime.strptime(match[0], '%Y-%m-%d %H:%M:%S,%f') + timedelta(seconds=1): int(match[1]) for match in out_matches}

In [82]:
df = data_packing.copy()
# df = df[df['isAccept'] == True]
df = df[['session_id', 'actionTime', 'isAccept', 'barCode', 'brand', 'reason']]
df['actionTime'] = df['actionTime'] - pd.Timedelta(seconds=1)

# df['actionTime'] = df['actionTime'] + pd.Timedelta(seconds=1)
# df['last_actionTime'] = df['actionTime'].shift(1)
# df.at[df.index[0], 'last_actionTime'] = "1999-10-11 00:00:00.000"
# df['last_actionTime'] = pd.to_datetime(df['last_actionTime'], format='%Y-%m-%d %H:%M:%S.%f')

df['next_actionTime'] = df['actionTime'].shift(-1)
df.at[df.index[-1], 'next_actionTime'] = df.at[df.index[-1], 'actionTime'] + timedelta(seconds=3)
df['next_actionTime'] = pd.to_datetime(df['next_actionTime'], format='%Y-%m-%d %H:%M:%S.%f')

df['passlineTime'] = df.apply(lambda row: between2Packing_addtime(row['actionTime'], row['next_actionTime'], in_dict, out_dict),  axis=1)
df['nb_in']  = df.apply(lambda row: between2Packing(row['actionTime'], row['next_actionTime'], in_dict),  axis=1)
df['nb_out'] = df.apply(lambda row: between2Packing(row['actionTime'], row['next_actionTime'], out_dict), axis=1)
df['isAcceptIA'] = df['nb_in'] - df['nb_out']
df.drop(columns=['next_actionTime'], inplace=True)

df['count_in']  = df['nb_in'].cumsum()
df['count_out'] = df['nb_out'].cumsum()

In [83]:
df[(df['session_id'] >= 1) & (df['session_id'] <= 1)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
1,1,2023-10-25 14:30:20.826135,True,3256223100141,U,,2023-10-25 14:30:21.505000,1,0,1,1,0
2,1,2023-10-25 14:30:24.864491,True,3254380008430,Cristaline,,2023-10-25 14:30:25.458000,1,0,1,2,0
3,1,2023-10-25 14:30:29.892241,True,3664320011510,Quezac,,2023-10-25 14:30:30.481000,1,0,1,3,0
4,1,2023-10-25 14:30:35.122441,True,3664320011510,Quezac,,2023-10-25 14:30:35.708000,1,0,1,4,0
5,1,2023-10-25 14:30:42.465418,True,3256223100141,U,,2023-10-25 14:30:43.072000,1,0,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
101,1,2023-10-25 14:45:16.436269,True,3256223100141,U,,2023-10-25 14:45:16.977000,1,0,1,97,1
102,1,2023-10-25 14:45:20.673822,True,3256223010563,U,,2023-10-25 14:45:21.337000,1,0,1,98,1
103,1,2023-10-25 14:45:25.089853,True,3256223100141,U,,2023-10-25 14:45:25.701000,1,0,1,99,1


In [84]:
df[(df['session_id'] == 2)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
106,2,2023-10-25 15:03:24.813193,True,3057640257773,Volvic,,2023-10-25 15:03:25.467000,1,0,1,102,1
107,2,2023-10-25 15:03:31.850200,True,3057640257773,Volvic,,2023-10-25 15:03:32.518000,1,0,1,103,1
108,2,2023-10-25 15:03:41.511452,True,3057640257773,Volvic,,2023-10-25 15:03:42.103000,1,0,1,104,1
109,2,2023-10-25 15:03:48.543609,True,3057640257773,Volvic,,2023-10-25 15:03:49.159000,1,0,1,105,1
110,2,2023-10-25 15:03:57.320232,True,3168930170549,Lipton,,2023-10-25 15:03:57.993000,1,0,1,106,1
111,2,2023-10-25 15:04:03.822178,True,3057640257773,Volvic,,2023-10-25 15:04:04.463000,1,0,1,107,1
112,2,2023-10-25 15:04:11.764667,True,3057640257773,Volvic,,2023-10-25 15:04:12.386000,1,0,1,108,1
113,2,2023-10-25 15:04:26.079958,True,3057640257773,Volvic,,2023-10-25 15:04:26.687000,1,0,1,109,1
114,2,2023-10-25 15:04:35.906370,True,3057640257773,Volvic,,2023-10-25 15:04:36.502000,1,0,1,110,1


In [85]:
df[(df['session_id'] == 3)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
119,3,2023-10-25 15:09:35.463482,True,7613036249928,Vittel,,2023-10-25 15:09:36.187000,1,0,1,115,1
120,3,2023-10-25 15:10:38.813530,True,7613036249928,Vittel,,2023-10-25 15:10:39.505000,1,0,1,116,1
121,3,2023-10-25 15:11:03.951214,True,7613036249928,Vittel,,2023-10-25 15:11:04.566000,1,0,1,117,1
122,3,2023-10-25 15:11:32.320853,True,7613036249928,Vittel,,0,0,0,0,117,1
123,3,2023-10-25 15:11:40.943718,True,7613036249928,Vittel,,2023-10-25 15:11:41.514000,1,0,1,118,1
124,3,2023-10-25 15:11:48.972158,True,7613036249928,Vittel,,2023-10-25 15:11:49.640000,1,0,1,119,1
125,3,2023-10-25 15:11:56.130595,True,7613036249928,Vittel,,2023-10-25 15:11:56.710000,1,0,1,120,1
126,3,2023-10-25 15:12:06.298974,True,7613036249928,Vittel,,2023-10-25 15:12:06.958000,1,0,1,121,1
127,3,2023-10-25 15:12:12.854725,True,7613036249928,Vittel,,2023-10-25 15:12:13.532000,1,0,1,122,1


In [86]:
df[(df['session_id'] == 4)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
137,4,2023-10-25 15:22:47.005590,True,3274080005003,Cristaline,,2023-10-25 15:22:47.449000,1,0,1,131,1
138,4,2023-10-25 15:23:15.093379,True,3256223510315,U,,2023-10-25 15:23:15.673000,1,0,1,132,1
139,4,2023-10-25 15:23:27.131309,True,3274080005003,Cristaline,,2023-10-25 15:23:27.734000,1,0,1,133,1
140,4,2023-10-25 15:23:39.394504,True,3274080005003,Cristaline,,2023-10-25 15:23:39.948000,1,0,1,134,1
141,4,2023-10-25 15:23:49.851483,True,4056489005599,Freeway,,2023-10-25 15:23:50.443000,1,0,1,135,1
142,4,2023-10-25 15:24:10.976895,True,3274080005003,Cristaline,,2023-10-25 15:24:11.594000,1,0,1,136,1
143,4,2023-10-25 15:24:27.243963,True,3274080005003,Cristaline,,2023-10-25 15:24:27.930000,1,0,1,137,1
144,4,2023-10-25 15:24:38.793663,True,3274080005003,Cristaline,,2023-10-25 15:24:39.505000,1,0,1,138,1
145,4,2023-10-25 15:24:49.172143,True,3186630002366,Loic Raison,,2023-10-25 15:24:49.828000,1,0,1,139,1


In [87]:
df[(df['session_id'] == 5)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
171,5,2023-10-25 15:32:25.955540,True,7613035833302,Perrier,,2023-10-25 15:32:26.425000,1,0,1,163,2
172,5,2023-10-25 15:32:32.805290,True,7613035833302,Perrier,,2023-10-25 15:32:33.416000,2,0,2,165,2
173,5,2023-10-25 15:32:46.117685,True,3564700371107,Leclerc,,0,0,0,0,165,2
174,5,2023-10-25 15:32:51.550773,True,3564700371107,Leclerc,,2023-10-25 15:32:51.936000,1,0,1,166,2
175,5,2023-10-25 15:32:57.068869,True,3274080005003,Cristaline,,2023-10-25 15:32:57.648000,1,0,1,167,2
176,5,2023-10-25 15:32:59.885816,True,3564700371107,Leclerc,,2023-10-25 15:33:00.562000,1,0,1,168,2
177,5,2023-10-25 15:33:04.571341,True,40882505,Saguaro,,2023-10-25 15:33:05.135000,1,0,1,169,2
178,5,2023-10-25 15:33:08.518064,True,40882505,Saguaro,,2023-10-25 15:33:09.092000,1,0,1,170,2
179,5,2023-10-25 15:33:11.536244,True,40882505,Saguaro,,2023-10-25 15:33:12.246000,1,0,1,171,2


In [88]:
df[(df['session_id'] == 6)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
221,6,2023-10-25 15:46:35.975897,True,7613036249928,Vittel,,2023-10-25 15:46:36.613000,1,0,1,212,3
222,6,2023-10-25 15:46:51.732197,True,3564700371107,Leclerc,,2023-10-25 15:46:52.357000,1,0,1,213,3
223,6,2023-10-25 15:47:02.187597,True,3564700371107,Leclerc,,2023-10-25 15:47:02.852000,1,0,1,214,3
224,6,2023-10-25 15:47:07.426902,True,3564700371107,Leclerc,,2023-10-25 15:47:08.032000,1,0,1,215,3
225,6,2023-10-25 15:47:12.555339,True,7613035833302,Perrier,,2023-10-25 15:47:13.210000,1,0,1,216,3
226,6,2023-10-25 15:47:21.290697,True,40882505,Saguaro,,2023-10-25 15:47:21.774000,1,0,1,217,3
227,6,2023-10-25 15:47:26.005013,True,7613036249928,Vittel,,2023-10-25 15:47:26.700000,1,0,1,218,3
228,6,2023-10-25 15:47:30.540292,True,4056489194446,Saguaro,,2023-10-25 15:47:31.153000,1,0,1,219,3
229,6,2023-10-25 15:47:37.943217,True,3564700371107,Leclerc,,2023-10-25 15:47:38.462000,1,0,1,220,3


In [89]:
df[(df['session_id'] == 7)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
255,7,2023-10-25 15:52:45.489431,True,3256223510315,U,,2023-10-25 15:52:46.134000,1,0,1,244,3
256,7,2023-10-25 15:52:50.203035,True,3256224059066,U,,2023-10-25 15:52:50.838000,1,0,1,245,3
257,7,2023-10-25 15:52:57.114320,True,3186630002366,Loic Raison,,2023-10-25 15:52:57.633000,1,0,1,246,3
258,7,2023-10-25 15:53:08.659517,True,3186630002366,Loic Raison,,0,0,0,0,246,3
259,7,2023-10-25 15:53:22.830727,True,3186630002366,Loic Raison,,2023-10-25 15:53:23.504000,1,0,1,247,3
260,7,2023-10-25 15:53:27.634240,True,3186630002366,Loic Raison,,2023-10-25 15:53:28.372000,1,0,1,248,3
261,7,2023-10-25 15:53:51.571169,True,3186630002366,Loic Raison,,2023-10-25 15:53:52.160000,1,0,1,249,3
262,7,2023-10-25 15:53:58.124276,True,3256223510315,U,,2023-10-25 15:53:58.776000,1,0,1,250,3
263,7,2023-10-25 15:54:05.502587,True,3256223510315,U,,2023-10-25 15:54:06.168000,1,0,1,251,3


In [90]:
df[(df['session_id'] == 8)]

,session_id,actionTime,isAccept,barCode,brand,reason,passlineTime,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
315,8,2023-10-25 16:02:05.071162,True,3274080005003,Cristaline,,2023-10-25 16:02:05.693000,1,0,1,301,3
316,8,2023-10-25 16:02:12.314187,True,3274080005003,Cristaline,,2023-10-25 16:02:12.992000,1,0,1,302,3
317,8,2023-10-25 16:02:15.833854,True,3250390145769,Ondine,,2023-10-25 16:02:16.475000,1,0,1,303,3
318,8,2023-10-25 16:02:27.583635,True,3274080005003,Cristaline,,2023-10-25 16:02:28.211000,1,0,1,304,3
319,8,2023-10-25 16:02:34.137589,True,3274080005003,Cristaline,,2023-10-25 16:02:34.649000,1,0,1,305,3
320,8,2023-10-25 16:02:39.455650,True,3274080005003,Cristaline,,2023-10-25 16:02:40.040000,1,0,1,306,3
321,8,2023-10-25 16:02:44.789690,True,3274080005003,Cristaline,,2023-10-25 16:02:45.420000,1,0,1,307,3
322,8,2023-10-25 16:02:50.324720,True,3274080005003,Cristaline,,2023-10-25 16:02:50.940000,1,0,1,308,3
323,8,2023-10-25 16:02:55.322729,True,3274080005003,Cristaline,,2023-10-25 16:02:55.995000,1,0,1,309,3


In [95]:
df[df['session_id'] == 1]['isAccept'].value_counts()

True    1
Name: isAccept, dtype: int64

In [24]:
df[df['session_id'] == 1]['nb_in'].sum(), df[df['session_id'] == 1]['nb_out'].sum()

(42, 0)

In [25]:
df['nb_out'].sum()

17

#### Merge according to Accepted actionTime

In [26]:
def between2Packing(start, stop, data_dict):
    count_id_list = [count for time, count in data_dict.items() if start <= time <= stop]
    if len(count_id_list) == 0:
        return 0
    else:
        return len(count_id_list)
    
last_non_zero = None

def update_count(row):
    global last_non_zero
    
    if row['count_out'] != 0:
        last_non_zero = row['count_out']
        return row['count_out']
    
    return last_non_zero if last_non_zero is not None else 0

In [27]:
df1 = data_packing.copy()
df1 = df1[df1['isAccept'] == True]
df1 = df1[['machineSerialNumber', 'session_id', 'actionTime', 'isAccept', 'barCode', 'brand', 'reason']]

In [28]:
df1['actionTime'] = df1['actionTime'] + pd.Timedelta(seconds=1)

df1['last_actionTime'] = df1['actionTime'].shift(1)
df1.at[df1.index[0], 'last_actionTime'] = "2000-01-01 00:00:00.000"
df1['last_actionTime'] = pd.to_datetime(df1['last_actionTime'], format='%Y-%m-%d %H:%M:%S.%f')

df1['nb_in']  = df1.apply(lambda row: between2Packing(row['last_actionTime'], row['actionTime'], in_dict),  axis=1)
df1['nb_out'] = df1.apply(lambda row: between2Packing(row['last_actionTime'], row['actionTime'], out_dict), axis=1)
df1['isAcceptIA'] = df1['nb_in'] - df1['nb_out']

df1['count_in']  = df1['nb_in'].cumsum()
df1['count_out'] = df1['nb_out'].cumsum()
# df1['count_in']  = df1.apply(lambda row: sumbetween2Packing(row['last_actionTime'], row['actionTime'], in_dict),  axis=1)
# df1['count_out'] = df1.apply(lambda row: sumbetween2Packing(row['last_actionTime'], row['actionTime'], out_dict),  axis=1)
# df1['count_out'] = df1.apply(update_count, axis=1)

df1.drop(columns=['last_actionTime'], inplace=True)

In [29]:
# df1[(df1['session_id'] >= 1) & (df1['session_id'] <= 4)]
# df1[(df1['session_id'] == 1)]
df1.iloc[0:60]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
1,00323,1,2023-10-20 15:51:29.547156,True,8024884500403,Courmayeur,,0,0,0,0,0
2,00323,1,2023-10-20 15:51:40.285810,True,8024884500403,Courmayeur,,0,0,0,0,0
3,00323,1,2023-10-20 15:51:48.086518,True,8024884500403,Courmayeur,,0,0,0,0,0
4,00323,1,2023-10-20 15:51:55.070732,True,3450970070782,Eco+,,1,0,1,1,0
5,00323,1,2023-10-20 15:52:00.999094,True,3450970070782,Eco+,,1,0,1,2,0
6,00323,1,2023-10-20 15:52:14.035877,True,8024884500403,Courmayeur,,1,0,1,3,0
7,00323,1,2023-10-20 15:52:22.117612,True,8024884500403,Courmayeur,,1,0,1,4,0
8,00323,1,2023-10-20 15:52:28.705912,True,8024884500403,Courmayeur,,1,0,1,5,0
9,00323,1,2023-10-20 15:52:36.068627,True,3450970070782,Eco+,,1,0,1,6,0


In [54]:
df1.iloc[60:120]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
62,00007,5,2023-08-26 08:42:34.148717,True,3274080005003,Cristaline,,1,0,1,63,2
63,00007,5,2023-08-26 08:42:41.662434,True,3274080005003,Cristaline,,1,0,1,64,2
64,00007,5,2023-08-26 08:42:48.612258,True,3660992012589,Prix Mini,,1,0,1,65,2
65,00007,5,2023-08-26 08:42:57.859265,True,3124480195319,May Tea,,1,0,1,66,2
66,00007,5,2023-08-26 08:43:08.809065,True,3254380008430,Cristaline,,1,0,1,67,2
67,00007,5,2023-08-26 08:43:35.587971,True,3068320123264,La Salvetat,,1,0,1,68,2
68,00007,5,2023-08-26 08:43:44.366976,True,3068320123264,La Salvetat,,1,0,1,69,2
69,00007,5,2023-08-26 08:43:53.379592,True,3274080005003,Cristaline,,1,0,1,70,2
70,00007,5,2023-08-26 08:44:02.484895,True,3274080005003,Cristaline,,1,0,1,71,2


In [55]:
df1.iloc[120:180]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
122,00007,7,2023-08-26 09:19:19.743908,True,3274080005003,Cristaline,,1,0,1,122,3
123,00007,7,2023-08-26 09:19:25.965468,True,3274080005003,Cristaline,,1,0,1,123,3
124,00007,7,2023-08-26 09:19:30.315162,True,3274080005003,Cristaline,,1,0,1,124,3
125,00007,7,2023-08-26 09:19:35.737496,True,3274080005003,Cristaline,,1,0,1,125,3
126,00007,7,2023-08-26 09:19:41.663035,True,3274080005003,Cristaline,,1,0,1,126,3
127,00007,8,2023-08-26 09:42:07.470089,True,3256223510100,U,,1,0,1,127,3
128,00007,8,2023-08-26 09:42:15.403131,True,3274080005003,Cristaline,,1,0,1,128,3
129,00007,8,2023-08-26 09:42:21.692966,True,3274080005003,Cristaline,,1,0,1,129,3
130,00007,8,2023-08-26 09:42:27.705314,True,3274080005003,Cristaline,,1,0,1,130,3


In [56]:
df1.iloc[180:240]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
184,00007,9,2023-08-26 09:49:33.291838,True,3257971309114,Mont Roucous,,1,0,1,181,3
185,00007,9,2023-08-26 09:49:37.318957,True,3123349014129,Joker,,1,0,1,182,3
186,00007,9,2023-08-26 09:49:41.288208,True,3257971309114,Mont Roucous,,1,0,1,183,3
187,00007,9,2023-08-26 09:49:44.999343,True,3608580110200,Andros,,1,0,1,184,3
188,00007,9,2023-08-26 09:50:19.869072,True,3256224762706,U,,1,0,1,185,3
189,00007,9,2023-08-26 09:50:25.527881,True,3256224762706,U,,0,0,0,185,3
190,00007,9,2023-08-26 09:50:37.554234,True,3274080005003,Cristaline,,2,0,2,187,3
191,00007,9,2023-08-26 09:50:43.280527,True,3124480195319,May Tea,,0,0,0,187,3
192,00007,9,2023-08-26 09:50:47.088212,True,3254380008430,Cristaline,,0,0,0,187,3


In [91]:
df1.iloc[240:300]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
246,00007,12,2023-08-26 12:11:48.704981,True,5000112611762,Coca-Cola,,2,0,2,251,17
248,00007,12,2023-08-26 12:12:10.777484,True,3274080005003,Cristaline,,2,1,1,253,18
249,00007,12,2023-08-26 12:12:17.596116,True,3274080005003,Cristaline,,1,0,1,254,18
250,00007,12,2023-08-26 12:12:24.670361,True,3274080005003,Cristaline,,1,0,1,255,18
251,00007,12,2023-08-26 12:12:31.583282,True,3274080005003,Cristaline,,1,0,1,256,18
252,00007,12,2023-08-26 12:12:43.215833,True,3274080005003,Cristaline,,1,0,1,257,18
253,00007,12,2023-08-26 12:12:52.172788,True,3274080005003,Cristaline,,1,0,1,258,18
254,00007,12,2023-08-26 12:13:00.002059,True,3274080005003,Cristaline,,1,0,1,259,18
255,00007,12,2023-08-26 12:13:07.294599,True,3274080005003,Cristaline,,1,0,1,260,18


In [92]:
df1.iloc[300:360]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
310,00007,14,2023-08-26 12:26:47.463456,True,20945589,Naturis,,1,0,1,314,20
311,00007,14,2023-08-26 12:26:52.279315,True,8410055050011,Font Vella,,1,0,1,315,20
312,00007,14,2023-08-26 12:26:57.591761,True,42399438,Minute Maid,,1,0,1,316,20
313,00007,15,2023-08-26 12:54:08.764205,True,3274080005003,Cristaline,,1,0,1,317,20
314,00007,15,2023-08-26 12:54:13.989472,True,3274080005003,Cristaline,,1,0,1,318,20
315,00007,15,2023-08-26 12:54:18.842543,True,3274080005003,Cristaline,,1,0,1,319,20
316,00007,15,2023-08-26 12:54:24.637447,True,3564700126790,Marque Repere,,1,0,1,320,20
317,00007,15,2023-08-26 12:54:28.066444,True,3274080005003,Cristaline,,1,0,1,321,20
318,00007,15,2023-08-26 12:54:31.867831,True,3274080005003,Cristaline,,1,0,1,322,20


In [94]:
df1.iloc[360:420]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
371,00007,18,2023-08-26 14:18:41.353012,True,3254380008430,Cristaline,,1,0,1,372,20
372,00007,18,2023-08-26 14:18:46.477977,True,3254380008430,Cristaline,,1,0,1,373,20
373,00007,18,2023-08-26 14:19:00.264906,True,3254380008430,Cristaline,,1,0,1,374,20
374,00007,18,2023-08-26 14:19:05.894128,True,3274080005003,Cristaline,,1,0,1,375,20
375,00007,18,2023-08-26 14:19:22.348745,True,3274080005003,Cristaline,,2,0,2,377,20
376,00007,18,2023-08-26 14:19:31.338004,True,3274080005003,Cristaline,,1,0,1,378,20
377,00007,18,2023-08-26 14:19:40.771309,True,5449000131836,Coca Cola,,1,0,1,379,20
378,00007,18,2023-08-26 14:19:44.383335,True,5449000000439,Coca-Cola,,1,0,1,380,20
379,00007,18,2023-08-26 14:19:49.105653,True,3068320118420,Badoit,,1,0,1,381,20


In [95]:
df1.iloc[420:480]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
432,00007,21,2023-08-26 15:12:14.364423,True,3274080005003,Cristaline,,1,0,1,434,21
433,00007,21,2023-08-26 15:12:24.800659,True,3274080005003,Cristaline,,0,0,0,434,21
434,00007,21,2023-08-26 15:12:30.936165,True,3274080005003,Cristaline,,1,0,1,435,21
435,00007,21,2023-08-26 15:12:40.791431,True,3274080005003,Cristaline,,1,0,1,436,21
436,00007,21,2023-08-26 15:12:44.889646,True,3274080005003,Cristaline,,1,0,1,437,21
437,00007,21,2023-08-26 15:12:48.606015,True,3274080005003,Cristaline,,1,0,1,438,21
438,00007,21,2023-08-26 15:13:08.974214,True,3249760000654,Orangina,,1,0,1,439,21
439,00007,21,2023-08-26 15:13:16.120534,True,3274080005003,Cristaline,,1,0,1,440,21
440,00007,21,2023-08-26 15:13:29.429818,True,3274080005003,Cristaline,,1,0,1,441,21


##### les traces où log IA correspond pas à trace de packing (dans les sessions de mauvaise concordance)

In [24]:
grouped_df1 = df1.groupby('session_id').agg({'isAccept': 'sum','isAcceptIA': 'sum'})
not_equal_sessions = grouped_df1[grouped_df1['isAccept'] != grouped_df1['isAcceptIA']].index
not_equal_sessions

Int64Index([5, 6, 8, 9, 10, 11, 12, 14, 18, 20, 21, 22, 23, 24, 28, 29, 31, 32,
            33, 34],
           dtype='int64', name='session_id')

In [25]:
err = df1[df1['session_id'].isin(not_equal_sessions)][(df1['isAcceptIA'] != 1)]
err

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
78,00007,5,2023-08-26 08:45:13.312112,True,3256227459078,U,,0,0,0,78,2
79,00007,5,2023-08-26 08:45:22.968163,True,3660992012589,Prix Mini,,2,0,2,80,2
87,00007,5,2023-08-26 08:46:36.803825,True,3254380008430,Cristaline,,0,0,0,87,2
91,00007,6,2023-08-26 08:51:48.107601,True,8024884500403,Courmayeur,,2,0,2,92,2
107,00007,6,2023-08-26 08:54:05.546880,True,3274080005003,Cristaline,,0,0,0,108,3
...,...,...,...,...,...,...,...,...,...,...,...,...
770,00007,34,2023-08-26 18:54:24.664080,True,3256223510100,U,,0,0,0,789,49
772,00007,34,2023-08-26 18:54:41.735932,True,3256223510100,U,,2,0,2,792,49
775,00007,34,2023-08-26 18:54:55.397653,True,3068320120256,Evian,,2,0,2,796,49


In [26]:
err['brand'].describe()

count            128
unique            29
top       Cristaline
freq              69
Name: brand, dtype: object

In [27]:
err['brand'].value_counts()

Cristaline                    69
Lipton                         7
U                              6
Volvic                         5
Courmayeur                     4
Carrefour                      3
Hepar                          3
Coca-Cola                      3
Saint Amand                    3
Joker                          3
Look                           2
San Pellegrino                 2
Fanta                          2
Sunny Delight                  1
St Yorre                       1
Minute Maid                    1
Laqueuille - Marque Repère     1
Tropicana                      1
S. Pellegrino                  1
Oasis                          1
Perrier                        1
La Salvetat                    1
Badoit                         1
Paquito                        1
Eva                            1
Prix Mini                      1
Orangina                       1
May Tea                        1
Evian                          1
Name: brand, dtype: int64

In [28]:
len(err[err['brand']=='Cristaline'])/len(err)

0.5390625

In [29]:
len(data_packing[data_packing['brand']=='Cristaline'])/len(data_packing)

0.5213849287169042

In [30]:
len(err[err['brand']=='Lipton'])/len(err)

0.0546875

In [31]:
len(data_packing[data_packing['brand']=='Lipton'])/len(data_packing)

0.024439918533604887

In [32]:
df1[(df1['isAcceptIA'] != 1)]

,machineSerialNumber,session_id,actionTime,isAccept,barCode,brand,reason,nb_in,nb_out,isAcceptIA,count_in,count_out
packing_id,,,,,,,,,,,,
25,00007,1,2023-08-25 19:36:34.238202,True,7613036249928,Vittel,,0,0,0,25,1
32,00007,1,2023-08-25 19:37:54.031789,True,7613036249928,Vittel,,2,0,2,33,1
78,00007,5,2023-08-26 08:45:13.312112,True,3256227459078,U,,0,0,0,78,2
79,00007,5,2023-08-26 08:45:22.968163,True,3660992012589,Prix Mini,,2,0,2,80,2
87,00007,5,2023-08-26 08:46:36.803825,True,3254380008430,Cristaline,,0,0,0,87,2
...,...,...,...,...,...,...,...,...,...,...,...,...
885,00007,38,2023-08-27 09:34:23.339167,True,3274080005003,Cristaline,,2,0,2,906,50
886,00007,38,2023-08-27 09:34:33.687175,True,3274080005003,Cristaline,,0,0,0,906,50
887,00007,38,2023-08-27 09:34:41.292841,True,3274080005003,Cristaline,,2,0,2,908,50


#### Merge according to SessionId